In [4]:
import tensorflow as tf

In [11]:
import os
import shutil
import random

# define source and destination paths
dataset_path=r'C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\PetImages'
train_path=r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train"
val_path=r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\val"

# define categories
categories=['Cat','Dog']

# split ratio
split_ratio=0.8

In [12]:
for category in categories:
    os.makedirs(os.path.join(train_path,category),exist_ok=True)
    os.makedirs(os.path.join(val_path,category),exist_ok=True)
    
    # get all images in category folder
    images=os.listdir(os.path.join(dataset_path,category))
    random.shuffle(images)
    
    # split data
    train_size=int(len(images)*split_ratio)
    train_images=images[:train_size]
    val_images=images[train_size:]
    
    # move images to train annd val folders
    for img in train_images:
        shutil.move(os.path.join(dataset_path,category,img),os.path.join(train_path,category,img))
    for img in val_images:
        shutil.move(os.path.join(dataset_path,category,img),os.path.join(val_path,category,img))
print("Dataset successfully split into train and validation sets!")

Dataset successfully split into train and validation sets!


In [20]:
import os 
from PIL import Image

# function to clean and  convert images
def clean_and_convert(directory):
    for  category in os.listdir(directory):
        category_path=os.path.join(directory,category)
        for  img_name in os.listdir(category_path):
            img_path=os.path.join(category_path,img_name)
            try:
                with Image.open(img_path) as  img:
                    # convert to RGB  and  save as JPEG if not already
                    if  img.format !="JPEG":
                        img=img.convert("RGB")
                        new_img_path=os.path.splitext(img_path)[0]+'jpg'
                        img.save(new_img_path,'JPEG')
                        os.remove(img_path) # remove the  old file
                        print(f"converted and  saved:{new_img_path}")
            except Exception as e:
                print(f"Removing corrupted or  unreadable image:{img_path} - {e}")
                os.remove(img_path)# remove corrupted or unreadable files

# clean both train and  val directories
clean_and_convert(train_path)
clean_and_convert(val_path)

print("All images cleaned and converted into  JPEG format")

converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Cat\10073jpg
Removing corrupted or  unreadable image:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Cat\10125.jpg - [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\hexdy\\OneDrive\\Desktop\\Batch_AI\\Deep_Learning\\train\\Cat\\10125.jpg'
Removing corrupted or  unreadable image:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Cat\10404.jpg - [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\hexdy\\OneDrive\\Desktop\\Batch_AI\\Deep_Learning\\train\\Cat\\10404.jpg'
Removing corrupted or  unreadable image:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Cat\10501.jpg - [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\hexdy\\OneDrive\\Desktop\\Batch_AI\\Deep_Learning\\train\\Cat\\10501.jpg'
Removing corru

c:\Users\hexdy\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removing corrupted or  unreadable image:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9188.jpg - [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\hexdy\\OneDrive\\Desktop\\Batch_AI\\Deep_Learning\\train\\Dog\\9188.jpg'
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9367jpg
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9414jpg
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9500jpg
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9556jpg
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9640jpg
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9643jpg
converted and  saved:C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\train\Dog\9851jpg
converted and  saved:C:\Users\hexdy\OneDrive\De

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image data generators(Augmentation)
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=20,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1./255)

# Load images
train_generator=train_datagen.flow_from_directory(train_path,target_size=(150,150),batch_size=32,class_mode="binary")
val_generator=val_datagen.flow_from_directory(val_path,target_size=(150,150),batch_size=32,class_mode="binary")

# define CNN model
model=Sequential([Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
                  MaxPooling2D(2,2),
                  Conv2D(64,(3,3),activation='relu'),
                  MaxPooling2D(2,2),
                  Conv2D(128,(3,3),activation='relu'),
                  MaxPooling2D(2,2),
                  Flatten(),
                  Dense(512,activation='relu'),
                  Dropout(0.5),
                  Dense(1,activation='sigmoid') # binary classification (cats vs dogs)
                  ])
# compile model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# train  model
model.fit(train_generator,validation_data=val_generator,epochs=2)

# save model
model.save("cat_dog_classifier.h5")
print("model training completed and saved!")


Found 19974 images belonging to 2 classes.
Found 4995 images belonging to 2 classes.


c:\Users\hexdy\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\hexdy\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 277s 442ms/step - accuracy: 0.5877 - loss: 0.6837 - val_accuracy: 0.7143 - val_loss: 0.5696
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 254s 407ms/step - accuracy: 0.7296 - loss: 0.5473 - val_accuracy: 0.7998 - val_loss: 0.4317


model training completed and saved!


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load trained model
model = tf.keras.models.load_model("cat_dog_classifier.h5")

# Load and preprocess test image
img_path = r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Deep_Learning\samples_for_prediction\dog_sample1.webp"
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
prediction

if prediction > 0.5:
    print("🐶 It's a Dog!")
else:
    print("🐱 It's a Cat!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
🐶 It's a Dog!
